In [3]:
import numpy as np
from scipy.special import hyp1f1, gamma


def R(l1l2, m1m2, n1n2, Cx, Cy, Cz, Px, Py, Pz, p, R1, Rbuffer, check=0):
    # check = 0, normal calculation. 
    # check = 1, derivative calculation
    
    PCx = Px-Cx
    PCy = Py-Cy
    PCz = Pz-Cz
    RPC = ((PCx)**2+(PCy)**2+(PCz)**2)**0.5
    if check == 0:
        for t in range(0, l1l2+1):
            for u in range(0, m1m2+1):
                for v in range(0, n1n2+1):
                    # Check the range of n, to ensure no redundent n are calculated
                    if t == u == 0:
                        exclude_from_n = v
                    elif t == 0:
                        exclude_from_n = n1n2 + u
                    else:
                        exclude_from_n = n1n2 + m1m2 + t
                    for n in range(0, l1l2+m1m2+n1n2+1-exclude_from_n):
                        val = 0.0
                        if t == u == v == 0:
                            Rbuffer[t,u,v,n] = (-2.0*p)**n*boys(n,p*RPC*RPC)
                        else:
                            if t == u == 0:
                                if v > 1:
                                    val += (v-1)*Rbuffer[t,u,v-2,n+1]
                                val += PCz*Rbuffer[t,u,v-1,n+1]  
                            elif t == 0:
                                if u > 1:
                                    val += (u-1)*Rbuffer[t,u-2,v,n+1]
                                val += PCy*Rbuffer[t,u-1,v,n+1]
                            else:
                                if t > 1:
                                    val += (t-1)*Rbuffer[t-2,u,v,n+1]
                                val += PCx*Rbuffer[t-1,u,v,n+1]
                            Rbuffer[t,u,v,n] = val
                            
                        if n == 0:
                            R1[t,u,v] = Rbuffer[t,u,v,n]
                            
                            
    elif check == 1:
        # For the first derivative +1 is needed in t, u and v
        # First the "normal" Rs are calculated
        for t in range(0, l1l2+1):
            for u in range(0, m1m2+1):
                for v in range(0, n1n2+1):
                    # Check the range of n, to ensure no redundent n are calculated
                    if t == u == 0:
                        exclude_from_n = v
                    elif t == 0:
                        exclude_from_n = n1n2 + u
                    else:
                        exclude_from_n = n1n2 + m1m2 + t
                    # +1 in n because of derivative    
                    for n in range(0, l1l2+m1m2+n1n2+1+1-exclude_from_n):
                        val = 0.0
                        if t == u == v == 0:
                            Rbuffer[t,u,v,n] = (-2.0*p)**n*boys(n,p*RPC*RPC)
                        else:
                            if t == u == 0:
                                if v > 1:
                                    val += (v-1)*Rbuffer[t,u,v-2,n+1]
                                val += PCz*Rbuffer[t,u,v-1,n+1]  
                            elif t == 0:
                                if u > 1:
                                    val += (u-1)*Rbuffer[t,u-2,v,n+1]
                                val += PCy*Rbuffer[t,u-1,v,n+1]
                            else:
                                if t > 1:
                                    val += (t-1)*Rbuffer[t-2,u,v,n+1]
                                val += PCx*Rbuffer[t-1,u,v,n+1]
                            Rbuffer[t,u,v,n] = val

                        if n == 0:
                            R1[t,u,v] = Rbuffer[t,u,v,n]
        
        # The next three blocks of code, calculates the
        # +1 incriments in the different angularmoment directions.
        # only one direction is +1 at a time.
        # eg. no need to calc R(t+1,u+1,v+1)
        # but only; R(t+1,u,v), R(t,u,v+1), R(t,u+,v)
        v = n1n2+1
        for t in range(0, l1l2+1):
            for u in range(0, m1m2+1):
                # Check the range of n, to ensure no redundent n are calculated
                if t == u == 0:
                    exclude_from_n = v
                elif t == 0:
                    exclude_from_n = n1n2 + 1 + u
                else:
                    exclude_from_n = n1n2 + 1 + m1m2 + t
                # +1 in n because of derivative    
                for n in range(0, l1l2+m1m2+n1n2+1+1-exclude_from_n):
                    val = 0.0
                    if t == u == 0:
                        if v > 1:
                            val += (v-1)*Rbuffer[t,u,v-2,n+1]
                        val += PCz*Rbuffer[t,u,v-1,n+1]  
                    elif t == 0:
                        if u > 1:
                            val += (u-1)*Rbuffer[t,u-2,v,n+1]
                        val += PCy*Rbuffer[t,u-1,v,n+1]
                    else:
                        if t > 1:
                            val += (t-1)*Rbuffer[t-2,u,v,n+1]
                        val += PCx*Rbuffer[t-1,u,v,n+1]
                    Rbuffer[t,u,v,n] = val

                    if n == 0:
                        R1[t,u,v] = Rbuffer[t,u,v,n]
        
        u = m1m2+1
        for t in range(0, l1l2+1):
            for v in range(0, n1n2+1):
                # Check the range of n, to ensure no redundent n are calculated
                if t == 0:
                    exclude_from_n = n1n2 + u
                else:
                    exclude_from_n = n1n2 + m1m2 + 1 + t
                # +1 in n because of derivative    
                for n in range(0, l1l2+m1m2+n1n2+1+1-exclude_from_n):
                    val = 0.0
                    if t == 0:
                        if u > 1:
                            val += (u-1)*Rbuffer[t,u-2,v,n+1]
                        val += PCy*Rbuffer[t,u-1,v,n+1]
                    else:
                        if t > 1:
                            val += (t-1)*Rbuffer[t-2,u,v,n+1]
                        val += PCx*Rbuffer[t-1,u,v,n+1]
                    Rbuffer[t,u,v,n] = val

                    if n == 0:
                        R1[t,u,v] = Rbuffer[t,u,v,n]

        t = l1l2+1
        for u in range(0, m1m2+1):
            for v in range(0, n1n2+1):
                # Check the range of n, to ensure no redundent n are calculated
                exclude_from_n = n1n2 + m1m2 + t
                # +1 in n because of derivative    
                for n in range(0, l1l2+m1m2+n1n2+1+1-exclude_from_n):
                    val = 0.0
                    if t > 1:
                        val += (t-1)*Rbuffer[t-2,u,v,n+1]
                    val += PCx*Rbuffer[t-1,u,v,n+1]
                    Rbuffer[t,u,v,n] = val

                    if n == 0:
                        R1[t,u,v] = Rbuffer[t,u,v,n]
        
    return R1
    

def boys(m, T):
    return hyp1f1(m+0.5,m+1.5,-T)/(2.0*m+1.0) 
    

def elelrep(p, q, l1, l2, l3, l4, m1, m2, m3, m4, n1, n2, n3, n4, N1, N2, N3, N4, c1, c2, c3, c4, E1, E2, E3, E4, E5, E6, Rpre):

    pi = 3.141592653589793238462643383279

    N = N1*N2*N3*N4*c1*c2*c3*c4
    
    val = 0.0
    for tau in range(l3+l4+1):
        for nu in range(m3+m4+1):
            for phi in range(n3+n4+1):
                factor = (-1.0)**(tau+nu+phi)
                for t in range(l1+l2+1):
                    for u in range(m1+m2+1):
                        for v in range(n1+n2+1):
                            val += E1[t]*E2[u]*E3[v]*E4[tau]*E5[nu]*E6[phi]*Rpre[t+tau,u+nu,v+phi]*factor

    val *= 2.0*pi**2.5/(p*q*(p+q)**0.5) 
    return val*N

def E(i, j, t, Qx, a, b, XPA, XPB, XAB):
    #McMurchie-Davidson scheme, 9.5.6 and 9.5.7 Helgaker

    p = a + b
    q = a*b/p
    if (t < 0) or (t > (i + j)):
        return 0.0
    elif i == j == t == 0:
        return np.exp(-q*Qx*Qx)
    elif j == 0:
        return (1.0/(2.0*p))*E(i-1,j,t-1,Qx,a,b,XPA,XPB,XAB) + XPA*E(i-1,j,t,Qx,a,b,XPA,XPB,XAB) + (t+1.0)*E(i-1,j,t+1,Qx,a,b,XPA,XPB,XAB)
    else:
        return (1.0/(2.0*p))*E(i,j-1,t-1,Qx,a,b,XPA,XPB,XAB) + XPB*E(i,j-1,t,Qx,a,b,XPA,XPB,XAB) + (t+1.0)*E(i,j-1,t+1,Qx,a,b,XPA,XPB,XAB)    


def ObarSaika_relation(ix, iy, iz, Theta, p, q, alpha, XPA, YPA, ZPA, XPQ, YPQ, ZPQ, Kab, Kcd, RPQ):

    pi = 3.141592653589793238462643383279
    
    # i's are the angularmomentum QM number of the first basisfunction
    # Theta is the general twointegrals
    
    Factor = 2.0*(pi)**(5.0/2.0)/(p*q*(p+q)**0.5)
    
    for z in range(0, iz+2):
        # +2 since electron transfer need +1
        for y in range(0 ,iy+2):
            for x in range(0, ix+2):
                if z == y == 0:
                    exclude_from_N = x
                elif z == 0:
                    exclude_from_N = ix+1 + y
                else:
                    exclude_from_N = ix+1 + iy+1 + z
                for N in range(0, ix+1+iy+1+iz+1 - exclude_from_N):
                    val = 0.0
                    
                    if x == y == z == 0:
                        val = Factor*Kab*Kcd*boys(N,alpha*RPQ*RPQ)
                    
                    elif z == y == 0:
                        if x > 0:
                            val += XPA*Theta[x-1,0,0,0,y,0,0,0,z,0,0,0,N]
                            val += -alpha/p*XPQ*Theta[x-1,0,0,0,y,0,0,0,z,0,0,0,N+1]
                        if x > 1:
                            val += x/(2.0*p)*(Theta[x-2,0,0,0,y,0,0,0,z,0,0,0,N] - alpha/p*Theta[x-2,0,0,0,y,0,0,0,z,0,0,0,N+1])
                            
                    elif y == 0:
                        if y > 0:
                            val += YPA*Theta[x,0,0,0,y-1,0,0,0,z,0,0,0,N]
                            val += -alpha/p*YPQ*Theta[x,0,0,0,y-1,0,0,0,z,0,0,0,N+1]
                        if y > 1:
                            val += y/(2.0*p)*(Theta[x,0,0,0,y-2,0,0,0,z,0,0,0,N] - alpha/p*Theta[x,0,0,0,y-2,0,0,0,z,0,0,0,N+1])

                        
                    else:
                        if z > 0:
                            val += ZPA*Theta[x,0,0,0,y,0,0,0,z-1,0,0,0,N]
                            val += -alpha/p*ZPQ*Theta[x,0,0,0,y,0,0,0,z-1,0,0,0,N+1]
                        if z > 1:
                            val += z/(2.0*p)*(Theta[x,0,0,0,y,0,0,0,z-2,0,0,0,N] - alpha/p*Theta[x,0,0,0,y,0,0,0,z-2,0,0,0,N+1])
                    
                    Theta[x,0,0,0,y,0,0,0,z,0,0,0,N] = val
    return Theta
                        
        

def electrontransfer(ix, iy, iz, kx, ky, kz, Theta, XAB, YAB, ZAB, XCD, YCD, ZCD, b, d, p, q):
    # k's are the angularmomentum QM number of the third basisfunction
    
    for z in range(0, kz+2):
        # +2 since horizontal relation need +1
        for y in range(0 ,ky+2):
            for x in range(0, kx+2):
                if x == y == z == 0:
                    None
                else:
                    val = 0.0
                    if z == y == 0:
                        if x > 0:
                            val += -(b*XAB+d*XCD)/q*Theta[ix,0,x-1,0,iy,0,y,0,iz,0,z,0,0]
                            val += -p/q*Theta[ix+1,0,x-1,0,iy,0,y,0,iz,0,z,0,0]
                            if ix > 0:
                                val += ix/(2.0*p)*Theta[ix-1,0,x-1,0,iy,0,y,0,iz,0,z,0,0]
                        if x > 1:
                            val += (x-1)/(2.0*q)*Theta[ix,0,x-2,0,iy,0,y,0,iz,0,z,0,0]
                            
                    elif y == 0:
                        if y > 0:
                            val += -(b*YAB+d*YCD)/q*Theta[ix,0,x,0,iy,0,y-1,0,iz,0,z,0,0]
                            val += -p/q*Theta[ix,0,x,0,iy+1,0,y-1,0,iz,0,z,0,0]
                            if iy > 0:
                                val += iy/(2.0*p)*Theta[ix,0,x,0,iy-1,0,y-1,0,iz,0,z,0,0]
                        if y > 1:
                            val += (y-1)/(2.0*q)*Theta[ix,0,x,0,iy,0,y-2,0,iz,0,z,0,0]
    
                    else:
                        if z > 0:
                            val += -(b*ZAB+d*ZCD)/q*Theta[ix,0,x,0,iy,0,y,0,iz,0,z-1,0,0]
                            val += -p/q*Theta[ix,0,x,0,iy,0,y,0,iz+1,0,z-1,0,0]
                            if iz > 0:
                                val += iz/(2.0*p)*Theta[ix,0,x,0,iy,0,y,0,iz-1,0,z-1,0,0]
                        if z > 1:
                            val += (z-1)/(2.0*q)*Theta[ix,0,x,0,iy,0,y,0,iz,0,z-2,0,0]
                        

                    Theta[ix,0,x,0,iy,0,y,0,iz,0,z,0,0] = val
    return Theta
                
    
    
def horizontal_relation(ix, iy, iz, jx, jy, jz, kx, ky, kz, lx, ly, lz, Theta, XAB, YAB, ZAB, XCD, YCD, ZCD):
    # j's are the angularmomentum QM number of the second basisfunction
    # l's are the angularmomentum QM number of the fourth basisfunction
    
    # First horizontal relation
    for z in range(0, jz+1):
        for y in range(0 ,jy+1):
            for x in range(0, jx+1):
                if x == y == z == 0:
                    None
                else:
                    val = 0.0
                    if z == y == 0:
                        if x > 0:
                            val += Theta[ix+1,x-1,kx,0,iy,y,ky,0,iz,z,kz,0,0]
                            val += XAB*Theta[ix,x-1,kx,0,iy,y,ky,0,iz,z,kz,0,0]
                    elif y == 0:
                        if y > 0:
                            val += Theta[ix,x,kx,0,iy+1,y-1,ky,0,iz,z,kz,0,0]
                            val += YAB*Theta[ix,x,kx,0,iy,y-1,ky,0,iz,z,kz,0,0]
                    else:
                        if z > 0:
                            val += Theta[ix,x,kx,0,iy,y,ky,0,iz+1,z-1,kz,0,0]
                            val += ZAB*Theta[ix,x,kx,0,iy,y,ky,0,iz,z-1,kz,0,0]

                    Theta[ix,x,kx,0,iy,y,ky,0,iz,z,kz,0,0] = val
    
    # Second horizontal relation
    for z in range(0, lz+1):
        for y in range(0 ,ly+1):
            for x in range(0, lx+1):
                if x == y == z == 0:
                    None
                else:
                    val = 0.0
                    if z == y == 0:
                        if x > 0:
                            val += Theta[ix,jx,kx+1,x-1,iy,jy,ky,y,iz,jz,kz,z,0]
                            val += XAB*Theta[ix,jx,kx,x-1,iy,jy,ky,y,iz,jz,kz,z,0]
                    elif y == 0:
                        if y > 0:
                            val += Theta[ix,jx,kx,x,iy,jy,ky+1,y-1,iz,jz,kz,z,0]
                            val += YAB*Theta[ix,jx,kx,x,iy,jy,ky,y-1,iz,jz,kz,z,0]
                    else:
                        if z > 0:
                            val += Theta[ix,jx,kx,x,iy,jy,ky,y,iz,jz,kz+1,z-1,0]
                            val += ZAB*Theta[ix,jx,kx,x,iy,jy,ky,y,iz,jz,kz+1,z-1,0]

                    Theta[ix,jx,kx,x,iy,jy,ky,y,iz,jz,kz,z,0] = val
    
    return Theta[ix,jx,kx,lx,iy,jy,ky,ly,iz,jz,kz,lz,0]

In [5]:
import numpy as np

def runMD(basisidx, basisfloat, basisint, ERI, E1arr, E2arr, E3arr, R1buffer, Rbuffer):

    # basisidx [number of primitives, start index in basisfloat and basisint]
    # basisfloat array of float values for basis, N, zeta, c, x, y, z 
    # basisint array of integer values for basis, l, m, n, atomidx
    
    for k in range(0, len(basisidx)):
        for l in range(k, len(basisidx)):
            calc  = 0.0
            calc2 = 0.0
            calc3 = 0.0
            for i in range(basisidx[k,1],basisidx[k,1]+basisidx[k,0]):
                for j in range(basisidx[l,1],basisidx[l,1]+basisidx[l,0]):
                    a  = basisfloat[i,1]
                    b  = basisfloat[j,1]
                    Ax = basisfloat[i,3]
                    Ay = basisfloat[i,4]
                    Az = basisfloat[i,5]
                    Bx = basisfloat[j,3]
                    By = basisfloat[j,4]
                    Bz = basisfloat[j,5]
                    l1 = basisint[i,0]
                    l2 = basisint[j,0]
                    m1 = basisint[i,1]
                    m2 = basisint[j,1]
                    n1 = basisint[i,2]
                    n2 = basisint[j,2]
                    
                    p   = a+b
                    Px  = (a*Ax+b*Bx)/p
                    Py  = (a*Ay+b*By)/p
                    Pz  = (a*Az+b*Bz)/p
                    
                    for t in range(l1+l2+1):
                        E1arr[k,l,i-basisidx[k,1],j-basisidx[l,1],t] = E(l1,l2,t,Ax-Bx,a,b,Px-Ax,Px-Bx,Ax-Bx)
                    for u in range(m1+m2+1):
                        E2arr[k,l,i-basisidx[k,1],j-basisidx[l,1],u] = E(m1,m2,u,Ay-By,a,b,Py-Ay,Py-By,Ay-By)
                    for v in range(n1+n2+1):
                        E3arr[k,l,i-basisidx[k,1],j-basisidx[l,1],v] = E(n1,n2,v,Az-Bz,a,b,Pz-Az,Pz-Bz,Az-Bz)

    # Run ERI
    for mu in range(0, len(basisidx)):
        for nu in range(mu, len(basisidx)):
            munu = mu*(mu+1)//2+nu
            for lam in range(0, len(basisidx)):
                for sig in range(lam, len(basisidx)):
                    lamsig = lam*(lam+1)//2+sig
                    if munu >= lamsig:
                        calc = 0.0
                        for i in range(basisidx[mu,1],basisidx[mu,1]+basisidx[mu,0]):
                            N1 = 1.0#basisfloat[i,0]
                            a  = basisfloat[i,1]
                            c1 = 1.0#basisfloat[i,2]
                            Ax = basisfloat[i,3]
                            Ay = basisfloat[i,4]
                            Az = basisfloat[i,5]
                            l1 = basisint[i,0]
                            m1 = basisint[i,1]
                            n1 = basisint[i,2]
                            for j in range(basisidx[nu,1],basisidx[nu,1]+basisidx[nu,0]):
                                N2 = 1.0#basisfloat[j,0]
                                b  = basisfloat[j,1]
                                c2 = 1.0#basisfloat[j,2]
                                Bx = basisfloat[j,3]
                                By = basisfloat[j,4]
                                Bz = basisfloat[j,5]
                                l2 = basisint[j,0]
                                m2 = basisint[j,1]
                                n2 = basisint[j,2]

                                p   = a+b
                                Px  = (a*Ax+b*Bx)/p
                                Py  = (a*Ay+b*By)/p
                                Pz  = (a*Az+b*Bz)/p
                                
                                E1 = E1arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E2 = E2arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E3 = E3arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]

                                for k in range(basisidx[lam,1],basisidx[lam,1]+basisidx[lam,0]):
                                    N3 = 1.0#basisfloat[k,0]
                                    c  = basisfloat[k,1]
                                    c3 = 1.0#basisfloat[k,2]
                                    Cx = basisfloat[k,3]
                                    Cy = basisfloat[k,4]
                                    Cz = basisfloat[k,5]
                                    l3 = basisint[k,0]
                                    m3 = basisint[k,1]
                                    n3 = basisint[k,2]
                                    for l in range(basisidx[sig,1],basisidx[sig,1]+basisidx[sig,0]):
                                        N4 = 1.0#basisfloat[l,0]
                                        d  = basisfloat[l,1]
                                        c4 = 1.0#basisfloat[l,2]
                                        Dx = basisfloat[l,3]
                                        Dy = basisfloat[l,4]
                                        Dz = basisfloat[l,5]
                                        l4 = basisint[l,0]
                                        m4 = basisint[l,1]
                                        n4 = basisint[l,2]
                                                                                    
                                        q   = c+d
                                        Qx  = (c*Cx+d*Dx)/q
                                        Qy  = (c*Cy+d*Dy)/q
                                        Qz  = (c*Cz+d*Dz)/q

                                        E4 = E1arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E5 = E2arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E6 = E3arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        
                                        alpha = p*q/(p+q)
                                        
                                        R1 = R(l1+l2+l3+l4, m1+m2+m3+m4, n1+n2+n3+n4, Qx, Qy, Qz, Px, Py, Pz, alpha, R1buffer, Rbuffer)
                                        calc += elelrep(p,q,l1, l2, l3, l4, m1, m2, m3, m4, n1, n2, n3, n4, N1, N2, N3, N4, c1, c2, c3, c4, E1, E2, E3, E4, E5, E6, R1)
                            
                        ERI[mu,nu,lam,sig] = ERI[nu,mu,lam,sig] = ERI[mu,nu,sig,lam] = ERI[nu,mu,sig,lam] = ERI[lam,sig,mu,nu] = ERI[sig,lam,mu,nu] = ERI[lam,sig,nu,mu] = ERI[sig,lam,nu,mu] = calc            
                                
    #END OF run ERI
    return ERI

In [6]:
import numpy as np

def runOS(basisidx, basisfloat, basisint, ERI, E1arr, E2arr, E3arr, Theta):
    # basisidx [number of primitives, start index in basisfloat and basisint]
    # basisfloat array of float values for basis, N, zeta, c, x, y, z 
    # basisint array of integer values for basis, l, m, n, atomidx
    
    for k in range(0, len(basisidx)):
        for l in range(k, len(basisidx)):
            calc  = 0.0
            calc2 = 0.0
            calc3 = 0.0
            for i in range(basisidx[k,1],basisidx[k,1]+basisidx[k,0]):
                for j in range(basisidx[l,1],basisidx[l,1]+basisidx[l,0]):
                    a  = basisfloat[i,1]
                    b  = basisfloat[j,1]
                    Ax = basisfloat[i,3]
                    Ay = basisfloat[i,4]
                    Az = basisfloat[i,5]
                    Bx = basisfloat[j,3]
                    By = basisfloat[j,4]
                    Bz = basisfloat[j,5]
                    l1 = basisint[i,0]
                    l2 = basisint[j,0]
                    m1 = basisint[i,1]
                    m2 = basisint[j,1]
                    n1 = basisint[i,2]
                    n2 = basisint[j,2]
                    
                    p   = a+b
                    Px  = (a*Ax+b*Bx)/p
                    Py  = (a*Ay+b*By)/p
                    Pz  = (a*Az+b*Bz)/p
                    
                    for t in range(l1+l2+1):
                        E1arr[k,l,i-basisidx[k,1],j-basisidx[l,1],t] = E(l1,l2,t,Ax-Bx,a,b,Px-Ax,Px-Bx,Ax-Bx)
                    for u in range(m1+m2+1):
                        E2arr[k,l,i-basisidx[k,1],j-basisidx[l,1],u] = E(m1,m2,u,Ay-By,a,b,Py-Ay,Py-By,Ay-By)
                    for v in range(n1+n2+1):
                        E3arr[k,l,i-basisidx[k,1],j-basisidx[l,1],v] = E(n1,n2,v,Az-Bz,a,b,Pz-Az,Pz-Bz,Az-Bz)
    
    # Run ERI
    for mu in range(0, len(basisidx)):
        for nu in range(mu, len(basisidx)):
            munu = mu*(mu+1)//2+nu
            for lam in range(0, len(basisidx)):
                for sig in range(lam, len(basisidx)):
                    lamsig = lam*(lam+1)//2+sig
                    if munu >= lamsig:
                        calc = 0.0
                        
                        # MAKE THETA contract
                        Theta_contracted = np.zeros(np.shape(Theta))
                        
                        # The four next loops are over the primitive basisfunctions,
                        # in the four contracted basisfunctions
                        for i in range(basisidx[mu,1],basisidx[mu,1]+basisidx[mu,0]):
                            N1 = basisfloat[i,0]
                            a  = basisfloat[i,1]
                            c1 = basisfloat[i,2]
                            Ax = basisfloat[i,3]
                            Ay = basisfloat[i,4]
                            Az = basisfloat[i,5]
                            l1 = basisint[i,0]
                            m1 = basisint[i,1]
                            n1 = basisint[i,2]
                            for j in range(basisidx[nu,1],basisidx[nu,1]+basisidx[nu,0]):
                                N2 = basisfloat[j,0]
                                b  = basisfloat[j,1]
                                c2 = basisfloat[j,2]
                                Bx = basisfloat[j,3]
                                By = basisfloat[j,4]
                                Bz = basisfloat[j,5]
                                l2 = basisint[j,0]
                                m2 = basisint[j,1]
                                n2 = basisint[j,2]

                                p   = a+b
                                Px  = (a*Ax+b*Bx)/p
                                Py  = (a*Ay+b*By)/p
                                Pz  = (a*Az+b*Bz)/p
                                
                                E1 = E1arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E2 = E2arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                E3 = E3arr[mu,nu,i-basisidx[mu,1],j-basisidx[nu,1]]
                                
                                Kab = E1[0]*E2[0]*E3[0]
                                XAB = Ax - Bx
                                YAB = Ay - By
                                ZAB = Az - Bz
                                
                                XPA = Px - Ax
                                YPA = Py - Ay
                                ZPA = Pz - Az

                                for k in range(basisidx[lam,1],basisidx[lam,1]+basisidx[lam,0]):
                                    N3 = basisfloat[k,0]
                                    c  = basisfloat[k,1]
                                    c3 = basisfloat[k,2]
                                    Cx = basisfloat[k,3]
                                    Cy = basisfloat[k,4]
                                    Cz = basisfloat[k,5]
                                    l3 = basisint[k,0]
                                    m3 = basisint[k,1]
                                    n3 = basisint[k,2]
                                    for l in range(basisidx[sig,1],basisidx[sig,1]+basisidx[sig,0]):
                                        N4 = basisfloat[l,0]
                                        d  = basisfloat[l,1]
                                        c4 = basisfloat[l,2]
                                        Dx = basisfloat[l,3]
                                        Dy = basisfloat[l,4]
                                        Dz = basisfloat[l,5]
                                        l4 = basisint[l,0]
                                        m4 = basisint[l,1]
                                        n4 = basisint[l,2]
                                                                                    
                                        q   = c+d
                                        Qx  = (c*Cx+d*Dx)/q
                                        Qy  = (c*Cy+d*Dy)/q
                                        Qz  = (c*Cz+d*Dz)/q

                                        E4 = E1arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E5 = E2arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        E6 = E3arr[lam,sig,k-basisidx[lam,1],l-basisidx[sig,1]]
                                        
                                        Kcd = E4[0]*E5[0]*E6[0]
                                        XCD = Cx - Dx
                                        YCD = Cy - Dy
                                        ZCD = Cz - Dz
                                        
                                        XPQ = Px - Qx
                                        YPQ = Py - Qy
                                        ZPQ = Pz - Qz
                                        
                                        alpha = p*q/(p+q)
                                        RPQ = ((XPQ)**2+(YPQ)**2+(ZPQ)**2)**0.5
                                        
                                        Theta = ObarSaika_relation(l1, m1, n1, Theta, p, q, alpha, XPA, YPA, ZPA, XPQ, YPQ, ZPQ, Kab, Kcd, RPQ)
                                        Theta = electrontransfer(l1, m1, n1, l3, m3, n3, Theta, XAB, YAB, ZAB, XCD, YCD, ZCD, b, d, p, q)
                                        Theta_contracted = np.add(Theta_contracted, Theta)
                                        # What about N and c ?
                                        
                        calc = horizontal_relation(l1, m1, n1, l2, m2, n2, l3, m3, n3, l4, m4, n4, Theta_contracted, XAB, YAB, ZAB, XCD, YCD, ZCD)
                                        
                        ERI[mu,nu,lam,sig] = ERI[nu,mu,lam,sig] = ERI[mu,nu,sig,lam] = ERI[nu,mu,sig,lam] = ERI[lam,sig,mu,nu] = ERI[sig,lam,mu,nu] = ERI[lam,sig,nu,mu] = ERI[sig,lam,nu,mu] = calc            
                                
    #END OF run ERI
    return ERI

In [15]:
basisidx = np.load('basisidx.npy')
basisint = np.load('basisint.npy')
basisfloat = np.load('basisfloat.npy')
maxA = np.max(basisint)+2
ERI = np.zeros((len(basisidx),len(basisidx),len(basisidx),len(basisidx)))
# Making array to save E
E1arr = np.zeros((len(basisidx),len(basisidx),np.max(basisidx[:,0]),np.max(basisidx[:,0]),np.max(basisint[:,0:3])*2+1))
E2arr = np.zeros((len(basisidx),len(basisidx),np.max(basisidx[:,0]),np.max(basisidx[:,0]),np.max(basisint[:,0:3])*2+1))
E3arr = np.zeros((len(basisidx),len(basisidx),np.max(basisidx[:,0]),np.max(basisidx[:,0]),np.max(basisint[:,0:3])*2+1))
# Array to store R values, only created once if created here
R1buffer = np.zeros((4*np.max(basisint)+1,4*np.max(basisint)+1,4*np.max(basisint)+1))
Rbuffer = np.zeros((4*np.max(basisint)+1,4*np.max(basisint)+1,4*np.max(basisint)+1,3*4*np.max(basisint)+1))
# Make Theta
Theta = np.zeros((maxA,maxA,maxA,maxA,maxA,maxA,maxA,maxA,maxA,maxA,maxA,maxA,3*maxA+1))
import time

In [16]:
start = time.time()
MD_ERI = runMD(basisidx, basisfloat, basisint, ERI, E1arr, E2arr, E3arr, R1buffer, Rbuffer)
print(time.time()-start)

4.003035068511963


In [ ]:
start = time.time()
OS_ERI = runOS(basisidx, basisfloat, basisint, ERI, E1arr, E2arr, E3arr, Theta)
print(time.time()-start)

In [14]:
np.max(np.abs(MD_ERI-OS_ERI))

406.46099742387202